In [ ]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

# Input location of parameter

In [ ]:
analysis = 'kernel_ridge_regression_outliers_included'
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    data_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/analyses/roi-roi_correl/addbs_vta_to_memory_net_maxima/linear_regression_addbs_vta_to_memory_net_maxima/prepared_data_sbc_consolidated.csv'
    out_dir = os.path.join(os.path.dirname(data_path), f'{analysis}')
    #out_dir = r'path to out dir here'
    
    print('I have set pathnames in the Mac style')
    print('I will save to :', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    data_path = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\addbs_vta_to_memory_net_maxima\linear_regression_addbs_vta_to_memory_net_maxima\prepared_data.csv'
    out_dir = rf'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\analyses\roi-roi_correl\addbs_vta_to_memory_net_maxima\linear_regression_addbs_vta_to_memory_net_maxima\{analysis}'
    #out_dir = r'path to out dir here'

    print('I have set pathnames in the Windows style')

## Import pre-prepared dataframe

In [ ]:
data_df = pd.read_csv(data_path)
data_df.tail(3)

## Clean the dataframe

In [ ]:
data_df = data_df.reset_index(drop=True)
try:
    data_df.pop('Unnamed: 0')
except:
    print('none to pop')
data_df = data_df.rename(columns={'% Change from baseline (ADAS-Cog11)': 'percent_change_adascog11'})#, '07_default': 'default', '02_somatomotor_seed': 'somatomotor', '03_dorsal_attention': 'dorsal', '01_visual_seed': 'visual', '04_ventral_attention': 'ventral', '05_limbic': 'limbic', '06_frontoparietal': 'frontoparietal'})
if np.sum(np.isnan(data_df)).any():
    data_df.dropna(inplace=True) #data_df.fillna(method='bfill', inplace=True)
    data_df.reset_index(inplace=True, drop=True)
display(data_df)


In [ ]:
from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled = scaler.fit_transform(data_df)
# data_df = pd.DataFrame(scaled)
data_df=(data_df-data_df.mean())/data_df.std()


In [ ]:
#Choose specific columns to assess
new_df = data_df.copy()
# columns_of_interest = [
#     'SBC', 'Thal. Ant. Med. L', 'PFC Mid. Vent.'
    
# ]
# columns_to_extract = ['percent_change_adascog11'] + columns_of_interest
# new_df = new_df.loc[:, columns_to_extract]
# data_df = new_df
data_df.head(2)

## Kernels

## Run a LOOC On a Kernel Ridge Regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge

def kernel_ridge_loo(df:pd.DataFrame, target:str, alpha:float, gamma:float, degree:int, kernel:str):
    y_pred = []
    for i in range(0, len(df[target].values.tolist())):
        # splitting dataframe into train and test set
        df_test = df.loc[[i]]
        df_train = df.drop(df.index[i])
        
        X_train = df_train.drop(target, axis=1)
        y_train = df_train[target]
        X_test = df_test.drop(target, axis=1)
        y_test = df_test[target]
        
        model = KernelRidge(kernel=kernel, alpha=alpha, gamma=gamma, degree=degree)
        model.fit(X_train, y_train)
        y_pred.append(model.predict(X_test)[0])
        
    outcome_dictionary = {}
    outcome_dictionary['actuals'] = df[target].values.tolist()
    outcome_dictionary['predictions'] = y_pred
    try:
        outcome_dictionary['loocv_prediction_r'], outcome_dictionary['loocv_prediction_p'] = pearsonr(outcome_dictionary['actuals'], outcome_dictionary['predictions'])
        outcome_dictionary['loocv_prediction_r2'] = outcome_dictionary['loocv_prediction_r']**2
        outcome_dictionary['loocv_mean_squared_error'] = np.sum( np.square( (outcome_dictionary['actuals'] - outcome_dictionary['predictions']))) / len(outcome_dictionary['predictions'])
        outcome_dictionary['loocv_root_mean_squared_error'] = np.sqrt(outcome_dictionary['loocv_mean_squared_error'])
        outcome_dictionary['lootcv_mean_absolute_error'] = np.sum(np.abs((outcome_dictionary['predictions'] - outcome_dictionary['actuals']))) / len(outcome_dictionary['predictions'])    
    except:
        print('Error generating outcome metrics')
    outcome_df = pd.DataFrame(outcome_dictionary)
    
    return outcome_df


In [ ]:
kernels = ['additive_chi2', 'polynomial', 'poly', 'rbf', 'cosine', 'laplacian', 'precomputed', 'sigmoid', 'linear', 'chi2']
kernel = 'polynomial'
loo_results = kernel_ridge_loo(df=data_df, target='percent_change_adascog11', kernel=kernel, alpha=.01, gamma=.0001, degree=2)
# loo_df = pd.DataFrame(loo_results)
from scipy.stats import pearsonr

#Calculate metrics
loo_results.dropna(inplace=True)
loo_results['loocv_pearson_r'], loo_results['loocv_pearson_p'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
loo_results['loocv_r_squared'] = np.square(loo_results['loocv_pearson_r'])
_, loo_results['loocv_pearson_p_greater'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='greater')
_, loo_results['loocv_pearson_p_lesser'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='less')
_, loo_results['loocv_pearson_p_two-sided'] = pearsonr(loo_results['predictions'], loo_results['actuals'], alternative='two-sided')
loo_results['loocv_mse'] = (np.sum(np.square((loo_results['actuals']-loo_results['predictions'])))) / len(loo_results['actuals'])
loo_results['loocv_rmse'] = np.sqrt(loo_results['loocv_mse'])
loo_results['mean_absolute_error'] = np.sum(np.absolute(loo_results['predictions']-loo_results['actuals']))/len(loo_results['predictions'])

display(loo_results)

In [ ]:
#Save if desired
save=True
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
        print('Made directory')
    try:
        loo_results.to_csv(os.path.join(out_dir, f'loocv_{kernel}_kernel_ridge_regression_{data_df.columns.values.tolist()[1:]}_statistically_significant_{loo_results["mean_absolute_error"].values.tolist[1]}.csv'))
    except:
        loo_results.to_csv(os.path.join(out_dir, f'ols_leave_one_out_full_formula.csv'))


print('Saved to: ' + out_dir)

## Call a Custom Kernel, Run a Ridge Regression, and Get Coefficients

In [ ]:
# Define the list of kernel functions
def linear_kernel(X1, X2, **kwargs):
    """Linear kernel function"""
    return np.matmul(X1, X2.transpose())

def polynomial_kernel(x1, x2, degree, c=1, **kwargs):
    return (np.dot(x1, x2) + c)**degree

def rbf_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.power(np.linalg.norm(x1 - x2), 2))

def gaussian_kernel(x1, x2, sigma, **kwargs):
    return np.exp(-np.power(np.linalg.norm(x1 - x2), 2) / (2 * sigma ** 2))

def sigmoid_kernel(x1, x2, k=1, c=1, **kwargs):
    return np.tanh(k * np.dot(x1, x2) + c)

def laplacian_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.linalg.norm(x1 - x2))

def exponential_kernel(x1, x2, gamma, **kwargs):
    return np.exp(-gamma * np.linalg.norm(x1 - x2))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from scipy.stats import pearsonr
kernel_list = ["linear_kernel", "polynomial_kernel", "rbf_kernel", "gaussian_kernel", "sigmoid_kernel", "laplacian_kernel", "exponential_kernel"] #, "exp", "log", "tanh"]
#Iterate over the kernels of interest

result_list_kernel = []; results_list_r2 = []; results_list_r = []; results_list_p = []; results_list_mse = []; results_list_rmse = []; results_list_mae = []
for kernel_string in kernel_list:

    #Generate Kernelized data
    n = data_df.shape[0]
    kernel = np.zeros((n,n))
    Y = data_df['percent_change_adascog11']
    X = data_df.drop('percent_change_adascog11', axis=1)
    sigma = 1000
    for i in range(n):
        for j in range(n):
            kernel[i,j] = eval(kernel_string)(X.iloc[i], X.iloc[j], gamma=1, sigma=1, k=1, degree=2)#, sigma=1)# degree=1)
            
    #Train a ridge regression on kernelized data
    model = KernelRidge(kernel='precomputed')
    results = model.fit(kernel[:, :], Y[:])

    results_dict = {}
    # results_dict['predictions'] = np.dot(kernel,model.dual_coef_)
    # results_dict['observations'] = Y[:]
    result_list_kernel.append(kernel_string)
    results_list_r2.append(results.score(kernel[:, :], Y[:]))
    r, p = pearsonr(np.dot(kernel,model.dual_coef_), Y[:])
    results_list_r.append(r); results_list_p.append(p)
    results_list_mse.append((np.sum(np.square((Y[:]-np.dot(kernel,model.dual_coef_))))) / len(Y[:]))
    results_list_rmse.append(np.sqrt((np.sum(np.square((Y[:]-np.dot(kernel,model.dual_coef_))))) / len(Y[:])))
    results_list_mae.append(np.sum(np.absolute(np.dot(kernel,model.dual_coef_)-Y[:]))/len(np.dot(kernel,model.dual_coef_)))
results_dict['kernel'] = result_list_kernel
results_dict['r2'] = results_list_r2; results_dict['r'] = results_list_r; results_dict['p'] = results_list_p
results_dict['mse'] = results_list_mse; results_dict['rmse'] = results_list_rmse; results_dict['mae'] = results_list_mae

kernel_comparison_kernel_ridge_results = pd.DataFrame(results_dict)
display(kernel_comparison_kernel_ridge_results)

In [ ]:
#Save Results if desired
save=True
if save:
    if os.path.isdir(out_dir) != True:
        os.makedirs(out_dir)
        print('Made directory')
    kernel_comparison_kernel_ridge_results.to_csv(os.path.join(out_dir, 'kernel_comparison_kernel_ridge_results.csv'))


print('Saved to: ' + out_dir)

## Run Kernels Manually

In [ ]:
n = data_df.shape[0]
kernel = np.zeros((n,n))
Y = data_df['percent_change_adascog11']
X = data_df.drop('percent_change_adascog11', axis=1)
sigma = 1000
for i in range(n):
    for j in range(n):
        kernel[i,j] = exponential_kernel(X.iloc[i], X.iloc[j], gamma=1, k=1, degree=1)#, sigma=1)# degree=1)
            

In [ ]:
#Generate Kernelized data
n = data_df.shape[0]
kernel = np.zeros((n,n))
Y = data_df['percent_change_adascog11']
X = data_df.drop('percent_change_adascog11', axis=1)
sigma = 1000
for i in range(n):
    for j in range(n):
        kernel[i,j] = exponential_kernel(X.iloc[i], X.iloc[j], gamma=1, k=1, degree=1)#, sigma=1)# degree=1)
        
#Train a ridge regression on kernelized data
model = KernelRidge(kernel='precomputed')
results = model.fit(kernel[:, :], Y[:])

results_dict = {}
results_dict['predictions'] = np.dot(kernel,model.dual_coef_)
results_dict['observations'] = Y[:]
results_dict['r2'] = results.score(kernel[:, :], Y[:])
results_dict['pearson_r'], results_dict['pearson_p'] = pearsonr(results_dict['predictions'], results_dict['observations'])
results_dict['mse'] = (np.sum(np.square((results_dict['observations']-results_dict['predictions'])))) / len(results_dict['observations'])
results_dict['rmse'] = np.sqrt(results_dict['mse'])
results_dict['mae'] = np.sum(np.absolute(results_dict['predictions']-results_dict['observations']))/len(results_dict['predictions'])

manual_kernel_ridge_results = pd.DataFrame(results_dict)
display(manual_kernel_ridge_results)


## Kernel Ridge Regression, K-Folds Cross Validation, Default Gaussian Kernel/

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from scipy.stats import pearsonr
import numpy as np

def kernel_ridge_kfold(df:pd.DataFrame, target:str, k:int, sigma:float):
    kf = KFold(n_splits=k, shuffle=True)
    y_pred = []
    y_test_all = []
    for train_index, test_index in kf.split(df):
        df_train, df_test = df.iloc[train_index], df.iloc[test_index]
        X_train = df_train.drop(target, axis=1)
        y_train = df_train[target]
        X_test = df_test.drop(target, axis=1)
        y_test = df_test[target]
        model = KernelRidge(kernel='rbf', gamma=1/(2*sigma**2))
        model.fit(X_train, y_train)
        y_pred.append(model.predict(X_test))
        y_test_all.append(y_test)
    y_test_all = np.concatenate(y_test_all)
    y_pred = np.concatenate(y_pred)
    R, P = pearsonr(y_pred, y_test_all)
    return R, P

R, P = kernel_ridge_kfold(df=data_df, target='percent_change_adascog11', k=2, sigma=100)#(data_df.shape[0]-1)/6)
results_df = pd.DataFrame({'R': R, 'P': P}, index=[0])
display(results_df)

save=False
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    results_df.to_csv(os.path.join(out_dir, 'kernel_ridge_regression_k_folds_results.csv'))


## Extract Coefficients from Ridge Regression

In [ ]:
def kernel_ridge(df:pd.DataFrame, target:str, sigma:float):
    y_pred = []
    y_test_all = []
    df_train = df.iloc[:, :]
    X_train = df_train.drop(target, axis=1)
    y_train = df_train[target]
    model = KernelRidge(kernel='rbf', gamma=1/(2*sigma**2))
    model.fit(X_train, y_train)
    print(model.predict(X_train))
    
    #Extract the coefficients
    kernel_space_coefficients = model.dual_coef_
    normal_coefficients = np.dot(X_train.transpose(), kernel_space_coefficients)
    return normal_coefficients, kernel_space_coefficients

coefficients, kernel_space_coefficients = kernel_ridge(df=data_df, target='percent_change_adascog11', sigma=(data_df.shape[0]-1)/6)


In [ ]:
coefficients_df = pd.DataFrame({'Coefficient Name': data_df.columns.to_list()[1:], 'Values': coefficients})
display(coefficients_df)

save=True
if save:
    if os.path.isdir(out_dir) == False:
        os.mkdir(out_dir)
    results_df.to_csv(os.path.join(out_dir, 'kernel_ridge_regression_coefficient_weight_results.csv'))


## Run initial model on all data

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

# Split dataframe into response and independent variables
X = data_df.iloc[:,1:].astype('float32')
y = data_df.iloc[:,0].astype('float32')

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a list of kernel functions
kernels = [tf.exp(-tf.reduce_sum(tf.square(X_train - X_train), axis=-1) / (2 * tf.square(0.1))),
           tf.matmul(X_train,tf.transpose(X_train)),
           tf.pow(tf.matmul(X_train,tf.transpose(X_train)),2),
           tf.pow(tf.matmul(X_train,tf.transpose(X_train)),3),
           tf.exp(tf.reduce_sum(X_train * X_train, axis=1)),
        #    tf.log(1+tf.reduce_sum(X_train * X_train, axis=1)),
           tf.tanh(tf.reduce_sum(X_train * X_train, axis=1))]
kernel_names = ["RBF-0.1", "Linear", "2nd degree polynomial", "3rd degree polynomial", "Exponential", "Logarithmic", "Hyperbolic tangent"]

# Create placeholders for the input data
X_ph = tf.compat.v1.placeholder(tf.float32, shape=(None, 2))
y_ph = tf.compat.v1.placeholder(tf.float32, shape=(None,))

# Create a dense layer
dense = tf.layers.Dense(units=1)

for kernel, kernel_name in zip(kernels, kernel_names):
    # Compute pairwise distances using the kernel trick
    kernel_output = kernel(X_ph, X_ph)
    kernel_output_train = kernel(X_ph, X_train)
    kernel_output_test = kernel(X_ph, X_test)

    # Pass the kernel output through the dense layer
    output = dense(kernel_output)
    output_train = dense(kernel_output_train)
    output_test = dense(kernel_output_test)

    # Define the loss function and optimizer
    loss = tf.losses.MeanAbsoluteError(y_ph, output)
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for _ in range(1000):
            _, train_loss = sess.run([train_op, loss], feed_dict={X_ph: X_train, y_ph: y_train})

        # Evaluate the model on the test set
        test_output = sess.run(output_test, feed_dict={X_ph: X_test})
        test_mse = tf.losses.mean_squared_error(y_test, test_output)
        print(f"Kernel: {kernel_name}, Test MSE: {test_mse}")

## Run LOOCV

In [ ]:
## Define Kernels

import tensorflow as tf
import numpy as np




In [ ]:
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import regularizers


def leave_one_out_cross_validation(df, kernels, kernel_names):
    # Split dataframe into response and independent variables
    X = df.iloc[:,1:].astype('float32')
    y = df.iloc[:,0].astype('float32')
    n = len(X)
    mae_list = []
    for i in range(n):
        X_test = X.iloc[i].values.reshape(1,-1)
        y_test = y.iloc[i]
        X_train = X.drop(i)
        y_train = y.drop(i)

        for kernel, kernel_name in zip(kernels, kernel_names):
            # Compute pairwise distances using the kernel trick
            if kernel_name == 'polynomial2':
                kernel_output = kernel(X_train, X_train, degree=2)
                kernel_output_test = kernel(X_test, X_train, degree=2)            
            if kernel_name == 'polynomial3':
                kernel_output = kernel(X_train, X_train, degree=3)
                kernel_output_test = kernel(X_test, X_train, degree=3)
            elif kernel_name == 'rbf':
                kernel_output = kernel(X_train, X_train, gamma=1/X_train.shape[1])
                kernel_output_test = kernel(X_test, X_train, gamma=1/X_train.shape[1])
            elif kernel_name == 'exp':
                kernel_output = kernel(X_train, X_train, gamma=1.0)
                kernel_output_test = kernel(X_test, X_train, gamma=1.0)
            elif kernel_name == 'log':
                kernel_output = kernel(X_train, X_train, alpha=1.0)
                kernel_output_test = kernel(X_test, X_train, alpha=1.0)
            elif kernel_name == 'tanh':
                kernel_output = kernel(X_train, X_train, alpha=1.0, beta=1.0)
                kernel_output_test = kernel(X_test, X_train, alpha=1.0, beta=1.0)
            else:
                kernel_output = kernel(X_train, X_train)
                kernel_output_test = kernel(X_test, X_train)

            print(kernel_output.shape, kernel_output_test.shape)

            # Define the model, regularization parameters, and optimizer
            dense = tf.keras.layers.Dense(units=1, kernel_regularizer=regularizers.l1(0.1))
            # loss = tf.losses.mean_absolute_error(y_train, output)
            optimizer = tf.optimizers.Adam()

            # training loop
            for epoch in range(1000):
                with tf.GradientTape() as tape:
                    logits = dense(tf.reshape(kernel_output, (-1, kernel_output.shape[1]))) #logits are just the output of dense layer, generally prior to activation layer
                    loss_value = tf.losses.mean_absolute_error(y_train, logits)
                grads = tape.gradient(loss_value, dense.trainable_variables)
                optimizer.apply_gradients(zip(grads, dense.trainable_variables))

            # Evaluate the model on the test set
            test_output = dense(kernel_output_test) #Runs the now trained dense model on the test set
            print(test_output, kernel_name, np.sum(np.isnan(test_output)))
            test_mae = tf.losses.mean_absolute_error(y_test, test_output)
            mae_list.append(test_mae)

    return mae_list

mae_list = leave_one_out_cross_validation(data_df, kernels, kernel_names)
print("MAE: ", mae_list)


In [ ]:
print("MAE: ", mae_list)

In [ ]:
import tensorflow as tf

# Define RBF kernel function
def rbf_kernel(X1, X2, gamma=1):
    """RBF (Radial Basis Function) kernel function"""
    sq_dists = tf.reduce_sum(X1**2, 1, keepdims=True) + tf.reduce_sum(X2**2, 1, keepdims=True) - 2 * tf.matmul(X1, X2, transpose_b=True)
    return tf.exp(-gamma * sq_dists)

# Define input data
X1 = tf.constant([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]], dtype=tf.float32)
X2 = tf.constant([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
                [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
                [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
                [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
                ], dtype=tf.float32)

# Pass input data through RBF kernel
kernel_output = rbf_kernel(X1, X2, gamma=0.0001)

# Pass kernel output through dense layer
dense = tf.keras.layers.Dense(units=1)
output = dense(kernel_output)

# Print prediction
print(output)
